In [1]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    model="gpt-5-mini",
    tools=[
        WebSearchTool(),
    ],
)


result = await Runner.run(agent, "Research on a few good reputable sources and write a one paragraph describing the relevance of MCP (model context protocol)")
print(result.final_output)

The Model Context Protocol (MCP), introduced by Anthropic on November 25, 2024, is an open‑source, model‑agnostic client–server standard that lets large language models access external files, services, and tools via a JSON‑RPC‑style interface (stdio/HTTP transports) to solve the “N×M” integration problem and standardize how models receive contextual data. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai)) Because MCP is permissively licensed and shipped with SDKs and reference servers, a quickly growing ecosystem (IDEs, agent frameworks, cloud services and enterprise connectors) has emerged and major vendors such as OpenAI, Microsoft and Google/DeepMind have announced support—making MCP a practical interoperability layer across models and platforms. ([developers.openai.com](https://developers.openai.com/codex/mcp/?utm_source=openai)) Its relevance is therefore high for developers and organizations that want reusable, “ship‑once” connectors and 

In [10]:
import json

from typing_extensions import Any
from pydantic import BaseModel, Field
from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Report(BaseModel):
    intro: str = Field(description="introduction to the topic of the report")
    arguments: str = Field(description="write 1 to 3 arguments to defend the researched thesis of the report")
    conclusion: str = Field(description="Write a small paragraph for the conclusion")
    sources: str = Field(description="List the sources (urls) used to write the report")

@function_tool
def write_file(ctx: RunContextWrapper[Any], path: str, contents: str, directory: str | None = None) -> str:
    """Write the contents to a file.

    Args:
        path: The path to the file to write.
        contents: The contents to write to the file.
        directory: The directory to write the file in (optional).
    """
    # In a real implementation, we'd write to the filesystem here.
    with open(path, "w") as f:
        f.write(contents)
    return f"File '{path}' written in directory '{directory}' with specified contents."

In [12]:
agent = Agent(
    name="Assistant",
    model="gpt-5-mini",
    tools=[write_file, WebSearchTool()],
    output_type=Report  
)

In [13]:
for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(tool.name)
        print(tool.description)
        print(json.dumps(tool.params_json_schema, indent=2))
        print()

write_file
Write the contents to a file.
{
  "properties": {
    "path": {
      "description": "The path to the file to write.",
      "title": "Path",
      "type": "string"
    },
    "contents": {
      "description": "The contents to write to the file.",
      "title": "Contents",
      "type": "string"
    },
    "directory": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "description": "The directory to write the file in (optional).",
      "title": "Directory"
    }
  },
  "required": [
    "path",
    "contents",
    "directory"
  ],
  "title": "write_file_args",
  "type": "object",
  "additionalProperties": false
}



In [16]:
result = await Runner.run(agent, "Search on at least 5 different sources and write a small report on the benefits of the model context protocol for agent development.")
print(result.final_output)

intro='The Model Context Protocol (MCP) is an open-source, application-layer standard introduced by Anthropic to let LLM hosts and external data/tools interoperate through a common client–server interface. By standardizing how models request and receive contextual data (files, tool outputs, database results, etc.), MCP reduces bespoke integrations and lets agent systems share memory, tools, and permissions in a predictable way. \ue200cite\ue202turn1search0\ue202turn1news16\ue201' arguments='1) Standardized interoperability and reduced integration cost — MCP turns the traditional M×N integration problem (many models × many data sources) into a manageable, standardized architecture: build MCP servers once and any MCP-capable host or agent can reuse them. That reduces duplicated engineering, testing, and maintenance across agent projects and speeds iteration. \ue200cite\ue202turn1search0\ue202turn0news22\ue201\n\n2) Persistent, shareable context that enables more capable agents — MCP give

In [17]:
from IPython.display import display, Markdown

def display_report(report: Report):
    """
    Formats the report output and displays it as Markdown in a notebook.
    """
    md_text = f"""
# Report

**Introduction**

{report.intro}

**Arguments**

{report.arguments}

**Conclusion**

{report.conclusion}

**Sources**

{report.sources}
"""
    display(Markdown(md_text))

display_report(result.final_output)


# Report

**Introduction**

The Model Context Protocol (MCP) is an open-source, application-layer standard introduced by Anthropic to let LLM hosts and external data/tools interoperate through a common client–server interface. By standardizing how models request and receive contextual data (files, tool outputs, database results, etc.), MCP reduces bespoke integrations and lets agent systems share memory, tools, and permissions in a predictable way. citeturn1search0turn1news16

**Arguments**

1) Standardized interoperability and reduced integration cost — MCP turns the traditional M×N integration problem (many models × many data sources) into a manageable, standardized architecture: build MCP servers once and any MCP-capable host or agent can reuse them. That reduces duplicated engineering, testing, and maintenance across agent projects and speeds iteration. citeturn1search0turn0news22

2) Persistent, shareable context that enables more capable agents — MCP gives agents a way to access up-to-date, structured context (shared memory, files, APIs) and to coordinate across tools and other agents; this supports longer-horizon tasks, multi-step workflows, and better relevance in decisions (e.g., multi-tool chains, code actions, or enterprise document lookups). Major platform players emphasize MCP’s role in creating interoperable, memory-enabled agents. citeturn1news16turn1news14turn1search6

3) Faster enterprise adoption and safer scale (with caveats) — MCP’s client/server model and available SDKs let enterprises expose internal systems (GitHub, Slack, databases, cloud services) to agents in a controlled way, enabling reusable tool registries, governed access, and faster productization of agent features. Several vendors and integrators (GitHub, Microsoft integrations, PwC, and Workato among others) are adopting MCP to reduce integration overhead and support governed deployments. At the same time, practitioners must design authentication, permissioning, and prompt-injection defenses because MCP surface area creates new security and privacy risks if left unmanaged. citeturn1search2turn0search1turn0news21turn0news22

**Conclusion**

Overall, MCP’s main benefits for agent development are predictable interoperability, reuse of integrations, and a practical route to persistent, multi-tool context that makes agents more capable and faster to build. Those gains have encouraged rapid adoption by platform and enterprise actors, but realizing them safely requires careful governance (auth, permissions, telemetry, and prompt-injection mitigations) as the ecosystem matures. citeturn1search0turn1news16turn0news22

**Sources**

https://github.com/modelcontextprotocol
https://www.theverge.com/2024/11/25/24305774/anthropic-model-context-protocol-data-sources
https://www.reuters.com/business/microsoft-wants-ai-agents-work-together-remember-things-2025-05-19/
https://docs.github.com/en/copilot/concepts/about-mcp
https://www.itpro.com/technology/artificial-intelligence/what-is-model-context-protocol-mcp
https://www.pwc.com/us/en/about-us/newsroom/press-releases/pwc-adds-support-for-mcp-in-agent-os.html
https://www.axios.com/sponsored/workato-delivers-industrys-first-enterprise-mcp-platform-for-ai-agents
https://en.wikipedia.org/wiki/Model_Context_Protocol
https://arize.com/blog/model-context-protocol/
